In [203]:
import re

import json
from typing import List, Dict, Tuple, Set, Any

import numpy as np
import numpy.typing as npt


# Define Constants


In [204]:
######################################################
### Constants                                      ###
######################################################
# Base Paths
INPUT_PATH = "./data"
MODEL_PATH = "./model"

# Model File names
VANILLA_MODEL_FILENAME = "vanillamodel.txt"
AVERAGED_MODEL_FILENAME = "averagedmodel.txt"

# Class Identifiers
TRUTHFUL = "True"
DECEPTIVE = "Fake"
POSITIVE = "Pos"
NEGATIVE = "Neg"

# File paths
TRAIN_FILE_PATH = f"{INPUT_PATH}/train-labeled.txt"
CLEANED_DATA_FILE_PATH = f"{INPUT_PATH}/cleaned-data.txt"
PREPROCESSED_DATA_FILE_PATH = f"{INPUT_PATH}/preprocessed-data.txt"

VANILLA_MODEL_FILE_PATH = f"{MODEL_PATH}/{VANILLA_MODEL_FILENAME}"
AVERAGED_MODEL_FILE_PATH = f"{MODEL_PATH}/{AVERAGED_MODEL_FILENAME}"

DEV_FILE_PATH = f"{INPUT_PATH}/dev-text.txt"
DEV_TAGGED_FILE_PATH = f"{INPUT_PATH}/dev-key.txt"

RANDOM_SEED = 42

DATA_COL = 3
SENTIMENT_TARGET_COL = 2
TRUTHFULNESS_TARGET_COL = 1


In [205]:
rng = np.random.default_rng(seed=RANDOM_SEED)
np.random.seed(RANDOM_SEED)


# Helper Functions


In [206]:
line = "07Zfn0z Fake Pos If you're looking for an elegant hotel in downtown Chicago, you have to stay here. The Ambassador East Hotel has very comfortable and beautiful large rooms and is like a home away from home. The perfect place for a business person, and if you have a small pet you can bring them too! I would give this place four stars and would definitely stay here again.\n"


In [207]:
input_regex = re.compile("(\w*) (\w*) (\w*) (.*)\n?")


In [208]:
re.match(input_regex, line).groups()


('07Zfn0z',
 'Fake',
 'Pos',
 "If you're looking for an elegant hotel in downtown Chicago, you have to stay here. The Ambassador East Hotel has very comfortable and beautiful large rooms and is like a home away from home. The perfect place for a business person, and if you have a small pet you can bring them too! I would give this place four stars and would definitely stay here again.")

In [209]:
data = []
data.append(re.match(input_regex, line).groups())
data


[('07Zfn0z',
  'Fake',
  'Pos',
  "If you're looking for an elegant hotel in downtown Chicago, you have to stay here. The Ambassador East Hotel has very comfortable and beautiful large rooms and is like a home away from home. The perfect place for a business person, and if you have a small pet you can bring them too! I would give this place four stars and would definitely stay here again.")]

In [210]:
np.array(data)[0, 1]


'Fake'

In [211]:
# Load Data
def load_data(input_file_path: str, type: str = "TRAIN") -> npt.NDArray:
    input_data = list()
    regex = "(\w*) (\w*) (\w*) (.*)\n?"
    if type == "TEST":
        regex = "(\w*) (.*)\n?"
    elif type == "LABELS":
        regex = "(\w*) (\w*) (\w*)\n?"
    input_regex = re.compile(regex)
    with open(input_file_path, mode="r") as input_file:
        for line in input_file:
            input_data.append(re.match(input_regex, line).groups())
    return np.array(input_data)


In [212]:
# Store Data
def store_data(date_file_path: str, data: npt.NDArray) -> None:
    with open(date_file_path, mode="w") as data_file:
        for row in data:
            data_file.write(f"{row[0]} {row[1]} {row[2]} {row[3]}\n")


In [213]:
# Store Model
def store_model(model_file_path: str, model_data: Any) -> None:
    # TODO: Need to check model_data
    with open(model_file_path, mode="w") as model_file:
        json.dump(model_data, model_file, ensure_ascii=False)


In [214]:
# Load Model
def load_model(model_file_path: str) -> npt.NDArray:
    # TODO: Extract data from JSON and return that only
    with open(model_file_path, mode="r") as model_file:
        model_data = json.load(model_file)
    return model_data


In [215]:
# Store Predictions
def store_predictions(output_file_path: str, predictions: List[Tuple[str, str, str]]) -> None:
    with open(output_file_path, mode="w") as output_file:
        for prediction in predictions:
            output_file.write(f"{prediction[0]} {prediction[1]} {prediction[2]}\n")


In [216]:
# Calculate Scores
def calculate_scores(y_true, y_pred, title: str):
    from sklearn.metrics import classification_report

    print(f"------------------------ {title} ------------------------")
    print(classification_report(y_true, y_pred))
    print("--------------------------------------------------------------------------------\n\n")


# Load Data


In [217]:
data = load_data(TRAIN_FILE_PATH, type="TRAIN")


# Data Cleaning


In [218]:
data[:, 3].shape


(960,)

In [219]:
# Convert all reviews to lower case (optional according to study)
def to_lower(data: npt.NDArray):
    n_data = data.shape[0]
    result = data.copy()
    for i in range(n_data):
        result[i] = result[i].lower()
    return result


In [220]:
def remove_html_encodings(data: npt.NDArray):
    import re

    n_data = data.shape[0]
    result = data.copy()
    for i in range(n_data):
        result[i] = re.sub(r"&#\d+;", " ", result[i])
    return result


In [221]:
def remove_html_tags(data: npt.NDArray):
    import re

    n_data = data.shape[0]
    result = data.copy()
    for i in range(n_data):
        result[i] = re.sub(r"<[a-zA-Z]+\s?/?>", "", result[i])
    return result


In [222]:
def remove_url(data: npt.NDArray):
    import re

    n_data = data.shape[0]
    result = data.copy()
    for i in range(n_data):
        result[i] = re.sub(r"https?://([\w\-\._]+){2,}/[\w\-\.\-/=\+_\?]+", "", result[i])
    return result


In [223]:
def remove_html_and_url(data):
    """Function to remove
             1. HTML encodings
             2. HTML tags (both closed and open)
             3. URLs

    Args:
        data (npt.NDArray): A Numpy Array of type string

    Returns:
        _type_: npt.NDArray
    """
    import re

    n_data = data.shape[0]
    result = data.copy()
    for i in range(n_data):
        # Remove HTML encodings
        result[i] = re.sub(r"&#\d+;", "", result[i])

        # Remove HTML tags (both open and closed)
        result[i] = re.sub(r"<[a-zA-Z]+\s?/?>", "", result[i])

        # Remove URLs
        result[i] = re.sub(r"https?://([\w\-\._]+){2,}/[\w\-\.\-/=\+_\?]+", "", result[i])

    return result


In [224]:
def replace_digits_with_tag(data: npt.NDArray):
    import re

    n_data = data.shape[0]
    result = data.copy()
    for i in range(n_data):
        result[i] = re.sub(r"\d+", " NUM ", result[i])
    return result


In [225]:
# Remove non-alphabetical characters
def remove_non_alpha_characters(data: npt.NDArray):
    import re

    n_data = data.shape[0]
    result = data.copy()
    for i in range(n_data):
        result[i] = re.sub(r"_+|\\|[^a-zA-Z0-9\s]", " ", result[i])
    return result


In [226]:
# Remove extra spaces
def remove_extra_spaces(data: npt.NDArray):
    import re

    n_data = data.shape[0]
    result = data.copy()
    for i in range(n_data):
        result[i] = re.sub(r"^\s*|\s\s*", " ", result[i])
    return result


In [227]:
# Install contractions package, if you don't have it
# ! pip install contractions


In [228]:
# Expanding contractions
def fix_contractions(data: npt.NDArray):
    import contractions

    def contraction_fixer(txt: str):
        return " ".join([contractions.fix(word) for word in txt.split()])

    n_data = data.shape[0]
    result = data.copy()
    for i in range(n_data):
        result[i] = contraction_fixer(result[i])
    return result


In [229]:
# A dictionary containing the columns and a list of functions to perform on it in order
data_cleaning_pipeline = {
    DATA_COL: [
        to_lower,
        remove_html_encodings,
        remove_html_tags,
        remove_url,
        fix_contractions,
        remove_non_alpha_characters,
        remove_extra_spaces,
    ]
}

cleaned_data = data.copy()

# Process all the cleaning instructions
for col, pipeline in data_cleaning_pipeline.items():
    # Get the column to perform cleaning on
    temp_data = cleaned_data[:, col].copy()

    # Perform all the cleaning functions sequencially
    for func in pipeline:
        print(f"Starting: {func.__name__}")
        temp_data = func(temp_data)
        print(f"Ended: {func.__name__}")

    # Replace the old column with cleaned one.
    cleaned_data[:, col] = temp_data.copy()


Starting: to_lower
Ended: to_lower
Starting: remove_html_encodings
Ended: remove_html_encodings
Starting: remove_html_tags
Ended: remove_html_tags
Starting: remove_url
Ended: remove_url
Starting: fix_contractions
Ended: fix_contractions
Starting: remove_non_alpha_characters
Ended: remove_non_alpha_characters
Starting: remove_extra_spaces
Ended: remove_extra_spaces


In [230]:
cleaned_data[:5, 3]


array([' if you are looking for an elegant hotel in downtown chicago you have to stay here the ambassador east hotel has very comfortable and beautiful large rooms and is like a home away from home the perfect place for a business person and if you have a small pet you can bring them too i would give this place four stars and would definitely stay here again ',
       ' the atmosphere at the talbott hotel is very welcoming when you first walk in the employees are friendly and will greet you when you first arrive the hotel seems to be luxurious even from the lobby and right away i was very excited for my stay i stayed in a double queen room with my friend the room is spacious enough and beds are very comfortable i did not try the room service but they do have a fitness club close by with a spa which is very convenient i also love how the room has individual climate controls very neat also handy that there is an ihome in the room i am usually stuck using just a regular alarm clock radio 

In [231]:
store_data(CLEANED_DATA_FILE_PATH, cleaned_data)


# Data Preprocessing


In [232]:
# Load Data
preprocessed_data = load_data(CLEANED_DATA_FILE_PATH)
preprocessed_data[0]


array(['07Zfn0z', 'Fake', 'Pos',
       ' if you are looking for an elegant hotel in downtown chicago you have to stay here the ambassador east hotel has very comfortable and beautiful large rooms and is like a home away from home the perfect place for a business person and if you have a small pet you can bring them too i would give this place four stars and would definitely stay here again '],
      dtype='<U4049')

In [233]:
def tokenize(data: npt.NDArray):
    import nltk
    from nltk.tokenize import word_tokenize

    nltk.download("punkt")

    n_data = data.shape[0]
    result = list()
    for i in range(n_data):
        result.append(word_tokenize(data[i]))
    return np.array(result)


In [234]:
def remove_stopwords(data: npt.NDArray):
    """Remove stop words using the NLTK stopwords dictionary

    Args:
        string (str): a document

    Returns:
        str: a document with stopwords removed
    """
    import nltk
    from nltk.corpus import stopwords

    nltk.download("stopwords")

    stopwords = set(stopwords.words())

    def remover(word_list: List[str], stopwords: Set[str]):
        return [word for word in word_list if not word in stopwords]

    n_data = data.shape[0]
    result = data.copy()
    for i in range(n_data):
        result[i] = remover(result[i], stopwords)
    return result


In [235]:
# Concatenate lemmatized sentences back into one sentence
def concatenate(data: npt.NDArray):
    n_data = data.shape[0]
    result = list()
    for i in range(n_data):
        result.append(" ".join(data[i]))
    return np.array(result)


In [236]:
# preprocessing_pipeline = {DATA_COL: [tokenize, remove_stopwords, concatenate]}

# # Run the pipeline
# preprocessed_data = cleaned_data.copy()

# # Process all the cleaning instructions
# for col, pipeline in preprocessing_pipeline.items():
#     # Get the column to perform cleaning on
#     temp_data = preprocessed_data[:, col].copy()

#     # Perform all the cleaning functions sequencially
#     for func in pipeline:
#         print(f"Starting: {func.__name__}")

#         temp_data = func(temp_data)

#         print(f"Ended: {func.__name__}")

#     # Replace the old column with cleaned one.
#     preprocessed_data[:, col] = temp_data.copy()


In [237]:
# store_data(PREPROCESSED_DATA_FILE_PATH, preprocessed_data)


In [238]:
# preprocessed_data[0]


# Split the Data


In [239]:
data = load_data(CLEANED_DATA_FILE_PATH)


In [240]:
# Split the data 80-20 split
from sklearn.model_selection import train_test_split

train, test = train_test_split(data, test_size=0.2, stratify=data[:, SENTIMENT_TARGET_COL], random_state=RANDOM_SEED)


In [241]:
train.shape, test.shape


((768, 4), (192, 4))

# Feature Extraction


## TF-IDF


In [242]:
import nltk
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer

nltk.download("punkt")

vectorizer = TfidfVectorizer(tokenizer=word_tokenize)
vectorizer.fit(data[:, DATA_COL])


[nltk_data] Downloading package punkt to /Users/aditya/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
/Users/aditya/.pyenv/versions/3.10.6/envs/csci544/lib/python3.10/site-packages/sklearn/feature_extraction/text.py:524: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


TfidfVectorizer(tokenizer=<function word_tokenize at 0x16cbfba30>)

# Perceptron Models


## 1. Perceptron


### A. Sentiment Classifier


In [243]:
from sklearn.linear_model import Perceptron

X_tfidf_train = vectorizer.transform(train[:, DATA_COL])
X_tfidf_test = vectorizer.transform(test[:, DATA_COL])
y_train = train[:, SENTIMENT_TARGET_COL]
y_test = test[:, SENTIMENT_TARGET_COL]

clf = Perceptron(max_iter=10, alpha=1, random_state=RANDOM_SEED, tol=1e-4, early_stopping=True)  # 0.45103975891511683

clf.fit(X_tfidf_train, y_train)

y_pred = clf.predict(X_tfidf_test)

calculate_scores(y_train, clf.predict(X_tfidf_train), title="Sentiment Analysis -- Train")
calculate_scores(y_test, y_pred, title="Sentiment Analysis -- Test")


------------------------ Sentiment Analysis -- Train ------------------------
              precision    recall  f1-score   support

         Neg       1.00      1.00      1.00       384
         Pos       1.00      1.00      1.00       384

    accuracy                           1.00       768
   macro avg       1.00      1.00      1.00       768
weighted avg       1.00      1.00      1.00       768

--------------------------------------------------------------------------------


------------------------ Sentiment Analysis -- Test ------------------------
              precision    recall  f1-score   support

         Neg       0.95      0.95      0.95        96
         Pos       0.95      0.95      0.95        96

    accuracy                           0.95       192
   macro avg       0.95      0.95      0.95       192
weighted avg       0.95      0.95      0.95       192

--------------------------------------------------------------------------------




### B. Truthfulness Classifier


In [244]:
from sklearn.linear_model import Perceptron

X_tfidf_train = vectorizer.transform(train[:, DATA_COL])
X_tfidf_test = vectorizer.transform(test[:, DATA_COL])
y_train = train[:, TRUTHFULNESS_TARGET_COL]
y_test = test[:, TRUTHFULNESS_TARGET_COL]

clf = Perceptron(max_iter=100, alpha=8e-3, random_state=RANDOM_SEED, early_stopping=True)  # 0.45103975891511683

clf.fit(X_tfidf_train, y_train)

y_pred = clf.predict(X_tfidf_test)

calculate_scores(y_train, clf.predict(X_tfidf_train), title="Truthfulness Analysis -- Train")
calculate_scores(y_test, y_pred, title="Truthfulness Analysis -- Test")


------------------------ Truthfulness Analysis -- Train ------------------------
              precision    recall  f1-score   support

        Fake       0.99      0.98      0.99       387
        True       0.98      0.99      0.99       381

    accuracy                           0.99       768
   macro avg       0.99      0.99      0.99       768
weighted avg       0.99      0.99      0.99       768

--------------------------------------------------------------------------------


------------------------ Truthfulness Analysis -- Test ------------------------
              precision    recall  f1-score   support

        Fake       0.82      0.83      0.82        93
        True       0.84      0.83      0.83        99

    accuracy                           0.83       192
   macro avg       0.83      0.83      0.83       192
weighted avg       0.83      0.83      0.83       192

--------------------------------------------------------------------------------




## 2. Averaged Perceptron


In [ ]:
## No Libs Available
